In [1]:
import json
from math import pi, tan
import numpy as np
import folium
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

#Flight height in m
flight_height = 10 #ATH þessi verður input

#Sensor width in inches
sensor_width = 1/2.4

#Converting sensor with to mm
sensor_width *= 25.4

#Focal length in mm
focal_length = 24 # ATH þurfum að finna þessa tölu betur

#Angle of view in degrees
angle_of_view = 2*np.arctan(sensor_width / (2*focal_length)) * (180/pi)

#Field of view in m
field_of_view = abs(2*(tan(angle_of_view/2)*flight_height))

print("field of view: ", field_of_view)

f = open('test_data_lyngby.json')
data = json.load(f)
x = []
y = []


for d in data:
    x.append(d['x'])
    y.append(d['y'])


max_x = max(x)
min_x = min(x)
max_y = max(y)
min_y = min(y)

x_cur = min_x
y_cur = min_y

grid_coordinates = []




new_tuple = [(x[idx],y[idx]) for idx in range(len(x))]
polygon = Polygon(new_tuple)
first = True
while(x_cur <= max_x):
    y_cur = min_y
    while(y_cur <= max_y):
        point2 = Point(x_cur ,y_cur)
        if polygon.contains(point2):
            grid_coordinates.append({"x":x_cur,"y":y_cur})
        y_cur += 0.8*(field_of_view/111139)
    x_cur += 0.8*(field_of_view/111139)

map = folium.Map(location=[55.78335044630297,12.494188681035387], zoom_start=100)
for point in grid_coordinates:
    if first:
        first = False
        folium.CircleMarker([point["x"], point["y"]], radius=1,color='blue').add_to(map)
    else:
        folium.CircleMarker([point["x"], point["y"]], radius=1,color='red').add_to(map)

map






field of view:  2.663347977287762


In [2]:
grid_coordinates

dist = np.sqrt(2*(0.8*(field_of_view/111139)**2))
print(dist)

3.0312476483796802e-05


In [3]:
# Python program for Dijkstra's single
# source shortest path algorithm. The program is
# for adjacency matrix representation of the graph
class Graph():
 
    def __init__(self, vertices):
        self.V = vertices
        self.graph = [[0 for column in range(vertices)]
                      for row in range(vertices)]
        print('GRAPH init: ',self.graph)
        print('V init: ',self.V)
 
    def printSolution(self, dist):
        print('GRAPH print: ',self.graph)
        print("Vertex \t Distance from Source")
        for node in range(self.V):
            print(node, "\t\t", dist[node])
 
    # A utility function to find the vertex with
    # minimum distance value, from the set of vertices
    # not yet included in shortest path tree
    def minDistance(self, dist, sptSet):
 
        # Initialize minimum distance for next node
        min = 1e7
 
        # Search not nearest vertex not in the
        # shortest path tree
        for v in range(self.V):
            if dist[v] < min and sptSet[v] == False:
                min = dist[v]
                min_index = v
 
        return min_index
 
    # Function that implements Dijkstra's single source
    # shortest path algorithm for a graph represented
    # using adjacency matrix representation
    def dijkstra(self, src):
 
        dist = [1e7] * self.V
        dist[src] = 0
        sptSet = [False] * self.V
 
        for cout in range(self.V):
 
            # Pick the minimum distance vertex from
            # the set of vertices not yet processed.
            # u is always equal to src in first iteration
            u = self.minDistance(dist, sptSet)
 
            # Put the minimum distance vertex in the
            # shortest path tree
            sptSet[u] = True
 
            # Update dist value of the adjacent vertices
            # of the picked vertex only if the current
            # distance is greater than new distance and
            # the vertex in not in the shortest path tree
            for v in range(self.V):
                if (self.graph[u][v] > 0 and
                   sptSet[v] == False and
                   dist[v] > dist[u] + self.graph[u][v]):
                    dist[v] = dist[u] + self.graph[u][v]
 
        self.printSolution(dist)
 


In [4]:
import networkx as nx
G = nx.Graph()

for idx, coordinate in enumerate(grid_coordinates):
    G.add_node(idx, is_full=0, coordinate=coordinate)
    
max_dist = np.sqrt(2*(0.8*(field_of_view/111139)**2))
G2 = G.copy()
for node in G:
    G2.remove_node(node)
    for node2 in G2:
        x_dist = abs(G.nodes[node]['coordinate']['x'] - G2.nodes[node2]['coordinate']['x'])
        y_dist = abs(G.nodes[node]['coordinate']['y'] - G2.nodes[node2]['coordinate']['y'])
        if (x_dist <= max_dist and y_dist <= max_dist):
            G.add_edge(node, node2, weight = np.sqrt(x_dist**2 + y_dist**2))
            #G.add_edge(node, node2)

In [5]:
import networkx.algorithms.approximation as nx_app

path = nx_app.traveling_salesman_problem(G, cycle=False)

In [6]:
list_of_coordinates = []
for node in path:
    list_of_coordinates.append(G.nodes[node]['coordinate'])

In [8]:
list_of_coordinates

[{'x': 55.783342468733196, 'y': 12.494245524363492},
 {'x': 55.783342468733196, 'y': 12.49422635307005},
 {'x': 55.783342468733196, 'y': 12.49420718177661}]

In [7]:
from folium.features import DivIcon

map = folium.Map(location=[55.647, 12.572], zoom_start=15)
for point in G.nodes:
    folium.CircleMarker([G.nodes[point]['coordinate']["x"], G.nodes[point]['coordinate']["y"]], radius=1,color='red').add_to(map)
    
    folium.map.Marker([G.nodes[point]['coordinate']["x"], G.nodes[point]['coordinate']["y"]],
                     icon=DivIcon(
                          icon_size=(30,30),
                          icon_anchor=(5,14),
                          html=f'<div style="font-size: 10pt">%s</div>' % str(point),
                      )
                     
                     ).add_to(map)

map